This notebook will be used to determine if I should be using Main Summary or Clients Daily for my Heavy User analysis.

Clients Daily is the standard so I'd prefer to use it, but I've found two issues with it:
* The Active Hours are sometimes way too high which skews the mean and standard deviation of Active Hours for the day.
* There are duplicate pings that are getting used in clients_daily, but this doesn't affect the summary stats much.

In [2]:
import pyspark.sql.functions as F

In [3]:
# Global Variables
sample_id = 42
day_1_date = '20180925'

Consider three different sets of data
* All daily pings
* Sum of daily pings over a day using same method as clients_daily
* Clients daily

In [5]:
ping_query = """
    SELECT
        client_id,
        coalesce(scalar_parent_browser_engagement_total_uri_count, 0) AS uri_count,
        scalar_parent_browser_engagement_active_ticks AS active_ticks,
        (coalesce(scalar_parent_browser_engagement_active_ticks, 0))*5/3600 AS active_hours,        
        subsession_length,
        (subsession_length/3600) AS subsession_hours,
        active_ticks as sm_active_ticks,
        session_length,
        profile_subsession_counter,
        subsession_counter,
        submission_date_s3,
        session_start_date,
        subsession_start_date,
        reason,
        active_addons_count,
        scalar_parent_browser_engagement_max_concurrent_tab_count AS tab_count,
        scalar_parent_browser_engagement_max_concurrent_window_count AS window_count,
        scalar_parent_browser_engagement_unique_domains_count AS domains_count,
        profile_creation_date,
        profile_reset_date,
        previous_build_id,
        normalized_channel,
        os,
        normalized_os_version,
        windows_build_number,
        install_year,
        creation_date,
        distribution_id,
        submission_date,
        app_build_id,
        app_version,
        update_channel,
        update_enabled,
        update_auto_download,
        active_experiment_branch,
        timezone_offset,
        vendor,
        is_default_browser,
        default_search_engine,
        devtools_toolbox_opened_count,
        client_submission_date,
        places_bookmarks_count,
        places_pages_count,
        scalar_content_telemetry_event_counts AS telem_event_counts,
        scalar_parent_browser_engagement_tab_open_event_count AS tab_event_count,
        scalar_parent_browser_engagement_window_open_event_count AS window_event_count,
        scalar_parent_browser_errors_collected_count AS errors_collected_count,
        scalar_parent_devtools_current_theme AS current_theme,
        scalar_parent_formautofill_availability AS formautofill_availability,
        scalar_parent_media_page_count AS media_page_count, 
        country,
        city,
        geo_subdivision1,
        locale,
        antivirus,
        antispyware,
        firewall,
        session_id,
        subsession_id,
        sync_configured,
        sync_count_desktop,
        sync_count_mobile,
        disabled_addons_ids,
        active_theme,
        user_prefs,
        experiments,
        sample_id,
        document_id
    FROM main_summary
    WHERE 
        app_name='Firefox'
        AND submission_date_s3 >= '{}'
        AND submission_date_s3 <= '{}'
        AND sample_id = '{}'
    ORDER BY
        client_id,
        submission_date_s3,
        profile_subsession_counter
    """

# From telemetry docs for how clients_daily determines values
sum_query = """
    SELECT 
        client_id,
        submission_date_s3,
        sum(coalesce(scalar_parent_browser_engagement_total_uri_count, 0)) AS td_uri,
        sum(coalesce(scalar_parent_browser_engagement_active_ticks, 0)) AS td_active_ticks,      
        sum(coalesce(scalar_parent_browser_engagement_active_ticks, 0)*5/3600) AS td_active_hours,
        sum(subsession_length/3600) AS td_subsession_hours,   
        sum(CASE WHEN subsession_counter = 1 THEN 1 ELSE 0 END) AS sessions_started_on_this_day,
        mean(active_addons_count) AS active_addons_count_mean,
        max(scalar_parent_browser_engagement_max_concurrent_tab_count) AS tab_count_max,
        max(scalar_parent_browser_engagement_max_concurrent_window_count) AS window_count_max,
        max(scalar_parent_browser_engagement_unique_domains_count) AS domains_count_max,
        first(profile_creation_date) AS profile_creation_date,
        first(previous_build_id) AS previous_build_id,
        first(normalized_channel) AS normalized_channel,
        first(os) AS os,
        first(normalized_os_version) AS normalized_os_version,
        first(windows_build_number) AS windows_build_number,
        first(install_year) AS install_year,
        first(distribution_id) AS distribution_id,
        count(distinct document_id) AS pings_aggregated_by_this_row,
        first(app_build_id) AS app_build_id,
        first(app_version) AS app_version,
        first(update_channel) AS update_channel,
        first(update_enabled) AS update_enabled,
        first(update_auto_download) AS update_auto_download,
        first(timezone_offset) AS timezone_offset,
        first(vendor) AS vendor,
        first(is_default_browser) AS is_default_browser,
        first(default_search_engine) AS default_search_engine,
        sum(devtools_toolbox_opened_count) AS devtools_toolbox_opened_count_sum,
        mean(places_bookmarks_count) AS places_bookmarks_count_mean,
        mean(places_pages_count) AS places_pages_count_mean,
        sum(scalar_parent_browser_engagement_tab_open_event_count) AS td_tab_event_count,
        sum(scalar_parent_browser_engagement_window_open_event_count) AS td_window_event_count,
        first(CASE WHEN country IS NOT NULL AND country != '??' THEN country ELSE NULL END) as country,
        first(CASE WHEN country IS NOT NULL AND country != '??' 
                  THEN CASE WHEN city IS NOT NULL THEN city ELSE '??' END
                  ELSE NULL END) AS city,
        first(geo_subdivision1) AS geo_subdivision1,
        first(locale) AS locale,
        first(sync_configured) AS sync_configured,
        sum(sync_count_desktop) AS sync_count_desktop,
        sum(sync_count_mobile) AS sync_count_mobile,
        first(sample_id) AS sample_id
    FROM main_summary
    WHERE 
        app_name='Firefox'
        AND submission_date_s3 >= '{}'
        AND submission_date_s3 <= '{}'
        AND sample_id = '{}'
    GROUP BY
        1, 2
    """

clients_daily = """
    SELECT
        client_id,
        submission_date_s3,
        coalesce(scalar_parent_browser_engagement_total_uri_count_sum, 0) AS uri_count_sum,     
        active_hours_sum,
        subsession_hours_sum,   
        sessions_started_on_this_day,
        active_addons_count_mean,
        scalar_parent_browser_engagement_max_concurrent_tab_count_max AS tab_count_max,
        scalar_parent_browser_engagement_max_concurrent_window_count_max AS window_count_max,
        scalar_parent_browser_engagement_unique_domains_count_max AS domains_count_max,
        profile_creation_date,
        previous_build_id,
        normalized_channel,
        os,
        normalized_os_version,
        windows_build_number,
        install_year,
        distribution_id,
        pings_aggregated_by_this_row,
        app_build_id,
        app_version,
        update_channel,
        update_enabled,
        update_auto_download,
        timezone_offset,
        vendor,
        is_default_browser,
        default_search_engine,
        devtools_toolbox_opened_count_sum,
        places_bookmarks_count_mean,
        places_pages_count_mean,
        scalar_parent_browser_engagement_tab_open_event_count_sum AS tab_event_count_sum,
        scalar_parent_browser_engagement_window_open_event_count_sum AS window_event_count_sum,
        country,
        city,
        geo_subdivision1,
        locale,
        sync_configured,
        sync_count_desktop_sum,
        sync_count_mobile_sum,
        sample_id
    FROM clients_daily
    WHERE 
        app_name='Firefox'
        AND submission_date_s3 >= '{}'
        AND submission_date_s3 <= '{}'
        AND sample_id = '{}'
    """

In [6]:
ms_1day_ping = spark.sql(ping_query.format(day_1_date, day_1_date, sample_id))
ms_1day_sum = spark.sql(sum_query.format(day_1_date, day_1_date,sample_id))
cd_1day = spark.sql(clients_daily.format(day_1_date,day_1_date,sample_id))

#### Compare the three count values of most interest: uri, active hours and subsession hours.  
  
The daily sum of pings and clients_daily are the same for uri count and subsession hours, but very different for active hours.

In [8]:
display(cd_1day['submission_date_s3', 'uri_count_sum', 'active_hours_sum', 'subsession_hours_sum'].describe())

summary,submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum
count,1045624,1045624,1045624,1045624
mean,2.0180925E7,131.88279438880517,0.8174494302508816,8.8460280000
stddev,0.0,4553.284675465032,16.100313423512116,74.1812503802097
min,20180925,0,0.0,0.000000
max,20180925,4530092,14137.222222222863,47504.499734


In [9]:
display(ms_1day_sum['submission_date_s3', 'td_uri', 'td_active_hours', 'td_subsession_hours'].describe())

summary,submission_date_s3,td_uri,td_active_hours,td_subsession_hours
count,1045624,1045624,1045624,1045624
mean,2.0180925E7,131.88279438880517,0.6433114580384508,8.846028020641818
stddev,0.0,4553.28467546504,1.465108961921598,74.18124579768515
min,20180925,0,0.0,0.0
max,20180925,4530092,436.93055555555543,47504.49972222209


In [10]:
ms_ping_cols = ms_1day_ping.columns
ms_ping_cols.remove('client_id')
ms_sum_cols = ms_1day_sum.columns
ms_sum_cols.remove('client_id')
cd_cols = cd_1day.columns
cd_cols.remove('client_id')

### High Active Hours

Why are the active hours so much higher in clients_daily than adding up the pings in main_summary?

In [12]:
# Other than 1 record, they all either have 0 or very high uri counts, and high to very high subsession_hours
cd_highah = cd_1day.where('active_hours_sum > 440')
display(cd_highah.select(cd_cols))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,0,616.64722222222,24.0,0,0.0,null,null,null,2018-06-27 00:00:00,null,release,Windows_NT,6.2,null,2013,null,1,20151216175450,null,release,true,true,-300,Mozilla,false,google,null,10.0,26.0,null,null,US,Chicago,IL,en-US,null,null,null,42
20180925,0,477.00833333333,24.999722,0,0.0,null,null,null,2015-10-09 00:00:00,null,release,Windows_NT,5.1,null,2015,null,1,20150929144111,null,release,true,true,120,Mozilla,true,google,null,8.0,46.0,null,null,ES,Zaragoza,AR,es-ES,null,null,null,42
20180925,0,7134.629167140291,8694.430577,420212,0.0,null,null,null,2017-11-19 00:00:00,null,release,Windows_NT,5.1,null,2013,null,107718,20151216175450,null,release,false,false,420,Mozilla,true,google,null,5.066666666666666,5.666666666666667,null,null,US,Pittsburgh,PA,en-US,null,null,null,42
20180925,73358,491.0680555557299,6380.579738,1195,9.576076555023924,316,11,48,2015-07-28 00:00:00,20160922113459,release,Windows_NT,6.1,7601,2015,null,1868,20171128222554,57.0.1,release,true,true,60,Mozilla,true,google,20,12.460591133004925,1801.6590361445783,8519,705,FR,??,BRE,fr,false,null,null,42
20180925,174113,799.4555555555197,3577.964996,662,4.461126005361931,81,15,61,2017-03-28 00:00:00,20170802111520,esr,Windows_NT,6.1,7601,2017,null,388,20170627155318,52.2.1,esr,false,false,-240,Mozilla,false,google-nocodes,13,15.715909090909092,8929.438636363637,45206,1600,US,New York,NY,en-US,false,null,null,42
20180925,0,756.82638888889,24.0,0,1.0,null,null,null,2016-02-02 00:00:00,null,release,Windows_NT,6.3,null,2014,null,1,20151216175450,null,release,true,true,120,Mozilla,false,google,null,8.0,51.0,null,null,PL,Warsaw,MZ,pl,null,null,null,42
20180925,165335,984.2125000002102,2771.188327,228,4.0,39,8,89,2018-02-26 00:00:00,null,esr,Windows_NT,6.1,null,2015,C3,156,20170417065206,52.1.0,esr,false,false,330,Mozilla,true,google,10,12.808219178082192,21826.130136986303,58993,815,US,Scottsdale,AZ,en-US,false,null,null,42
20180925,41980,1201.1472222233597,9665.515279,7479,1.9081752779594505,33,5,41,2016-04-14 00:00:00,20160316151906,esr,Windows_NT,6.1,7601,2016,yahoobr,7625,20160316151906,60.0.2,esr,false,false,-180,Mozilla,false,google,1025,28.794930875576036,5650.209101382488,4944,668,BR,Curitiba,PR,pt-BR,false,90,1,42
20180925,20842,613.8236111112802,1024.274449,239,5.921875,9,2,42,2014-05-16 00:00:00,20180621121604,release,Windows_NT,6.1,7601,2014,null,243,20150929144111,52.0.1,release,false,true,-240,Mozilla,false,other-Search,null,1.9478260869565218,2550.1052631578946,850,26,US,New York,NY,en-US,false,null,null,42
20180925,0,1002.6486111111101,22.998888,0,0.0,null,null,null,2012-09-28 00:00:00,null,release,Windows_NT,5.1,null,2012,null,7,20160105164030,null,release,false,false,180,Mozilla,true,yandex,null,3.2857142857142856,277.14285714285717,null,null,BY,Minsk,HM,ru,null,null,null,42


In [13]:
# There are only 26 records in clients_daily at this high end
display(cd_highah.select(cd_cols).describe())

summary,submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,timezone_offset,vendor,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
count,26,26,26,26,26,26,10,10,9,26,10,26,26,26,7,26,4,26,26,10,26,26,26,26,8,25,25,9,9,26,26,26,26,3,3,26
mean,2.0180925E7,31574.76923076923,1486.0751068559143,3795.8677800000,16896.96153846154,2.128775389391353,72.5,6.7,42.44444444444444,null,2.01647427150182E13,null,null,5.934615384615383,7601.0,2014.3846153846155,null,4916.2307692307695,2.015777554509973E13,56.5,null,174.23076923076923,null,null,145.625,16.07561159014342,4273.022669890093,17669.0,2372.5555555555557,null,null,22.5,null,37.333333333333336,0.3333333333333333,42.0
stddev,0.0,61296.008864726384,2881.8892043504975,9373.05021327815,82280.14039000214,2.7530592116088615,107.02777417308296,4.321779468896785,28.355383576629286,null,1.423101485425602E10,null,null,0.42042103438558626,0.0,1.6988684016585236,null,21054.48197473914,9.04132931400877E9,7.7781745930520225,null,268.5989289170465,null,null,356.1540346936581,11.68895610107829,6560.826957761941,20618.54686926312,3952.811692678741,null,null,6.363961030678928,null,46.30694692304097,0.5773502691896257,0.0
min,20180925,0,457.25833333333,22.998888,0,0.0,1,1,1,2011-03-28 00:00:00,20141126041045,esr,Windows_NT,5.1,7601,2010,C3,1,20150917150946,51.0,esr,-300,Mozilla,baidu,1,1.9478260869565218,5.666666666666667,850,26,BR,??,18,de,3,0,42
max,20180925,215225,14137.222222222863,47504.499734,420212,9.576076555023924,316,15,89,2018-06-27 00:00:00,20180830143136,release,Windows_NT,6.3,7601,2017,yahoobr,107718,20180920131237,62.0.2,release,540,Mozilla,yandex,1025,54.086021505376344,21826.130136986303,58993,12087,VN,Zaragoza,SP,zh-CN,90,1,42


Look at pings with 0 uri count  
  
**Where active_ticks are 0 in the pings, the active_hours_sum in clients_daily is high**

In [15]:
# One ping for client 99a0 with 0 active ticks
display(ms_1day_ping.select(ms_ping_cols).where("client_id like '%dea0c29499a0'"))

uri_count,active_ticks,active_hours,subsession_length,subsession_hours,sm_active_ticks,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_version,update_channel,update_enabled,update_auto_download,active_experiment_branch,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,telem_event_counts,tab_event_count,window_event_count,errors_collected_count,current_theme,formautofill_availability,media_page_count,country,city,geo_subdivision1,locale,antivirus,antispyware,firewall,session_id,subsession_id,sync_configured,sync_count_desktop,sync_count_mobile,disabled_addons_ids,active_theme,user_prefs,experiments,sample_id,document_id
0,null,0.0,86400,24.0,544915,25528816,301,297,20180925,2017-12-04T00:00:00.0+01:00,2018-09-25T00:00:00.0+02:00,daily,1,null,null,null,16833,null,null,release,Windows_NT,6.3,null,2014,2018-09-25T22:00:00.030Z,null,20180925,20151216175450,43.0.1,release,true,true,null,120,Mozilla,false,google,null,null,8,51,null,null,null,null,null,null,null,PL,Warsaw,MZ,pl,null,null,null,f83f6631-a6aa-459d-920c-c80da23b3a25,2ffd0c7b-54f4-48dc-9b46-149a9b906f78,null,null,null,null,"List({972ce4c6-7e08-4474-a285-3208198ce6fd}, false, Domyślny, false, false, 43.0.1, 4, null, false, false, 16688, 16833, null, null, null, null)",null,null,42,aa676488-1098-43be-87af-9668e15faa3c


In [16]:
# Clients daily for client 99a0  - why are there 756 active hours?
display(cd_1day.select(cd_cols).where("client_id like '%dea0c29499a0'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,0,756.82638888889,24.0,0,1.0,null,null,null,2016-02-02 00:00:00,null,release,Windows_NT,6.3,null,2014,null,1,20151216175450,43.0.1,release,true,true,120,Mozilla,false,google,null,8.0,51.0,null,null,PL,Warsaw,MZ,pl,null,null,null,42


In [17]:
# One ping for client 211c with 0 active ticks
display(ms_1day_ping.select(ms_ping_cols).where("client_id like '%f971bd94211c'"))

uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_version,update_channel,update_enabled,update_auto_download,active_experiment_branch,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,telem_event_counts,tab_event_count,window_event_count,errors_collected_count,current_theme,formautofill_availability,media_page_count,country,city,geo_subdivision1,locale,antivirus,antispyware,firewall,session_id,subsession_id,sync_configured,sync_count_desktop,sync_count_mobile,disabled_addons_ids,active_theme,user_prefs,experiments,sample_id,document_id
0,null,0.0,86400,24.0,3332887,494,39,20180925,2018-08-17T00:00:00.0-03:00,2018-09-24T00:00:00.0-03:00,daily,0,null,null,null,15685,null,null,release,Windows_NT,6.1,null,2012,2018-09-25T03:00:00.022Z,null,20180925,20151216175450,43.0.1,release,false,false,null,-180,Mozilla,true,google,null,null,22,5718,null,null,null,null,null,null,null,UY,Montevideo,MO,en-GB,null,null,null,20d5cd19-7cd6-4e6a-aea8-c1d556d32a30,4e657171-a606-41e0-a7ff-186416773598,null,null,null,null,"List({972ce4c6-7e08-4474-a285-3208198ce6fd}, false, Default, false, false, 43.0.1, 4, null, false, false, 15685, 17298, null, null, null, null)",null,null,42,f3c18eb5-2ad3-4a55-ac3f-02b79ce6849b


In [18]:
# Clients daily for client 211c - why are there 470 active hours?
display(cd_1day.select(cd_cols).where("client_id like '%f971bd94211c'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,0,470.83472222222,24.0,0,0.0,null,null,null,2012-12-11 00:00:00,null,release,Windows_NT,6.1,null,2012,null,1,20151216175450,43.0.1,release,false,false,-180,Mozilla,true,google,null,22.0,5718.0,null,null,UY,Montevideo,MO,en-GB,null,null,null,42


In [19]:
# Summary stats for all pings for client 9a4d - there are 153 pings with all 0 active_ticks
display(ms_1day_ping.select(ms_ping_cols).where("client_id like '%a749453b9a4d'").describe())

summary,uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_version,update_channel,active_experiment_branch,timezone_offset,vendor,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,tab_event_count,window_event_count,errors_collected_count,media_page_count,country,city,geo_subdivision1,locale,session_id,subsession_id,sync_count_desktop,sync_count_mobile,sample_id,document_id
count,153,0,153,153,153,153,153,153,153,153,153,153,153,0,0,0,153,0,0,153,153,153,0,153,153,0,153,153,153,153,0,153,153,153,0,0,71,71,0,0,0,0,153,153,153,153,153,153,0,0,153,153
mean,0.0,null,0.0,19275.843137254902,5.354400871459696,22458.483660130718,575.3202614379085,1.1372549019607843,2.0180925E7,null,null,null,1.0,null,null,null,15061.0,null,null,null,null,5.1,null,2016.0,null,null,2.0180925E7,2.015121617545E13,null,null,null,240.0,null,null,null,null,16.056338028169016,25.760563380281692,null,null,null,null,null,null,null,null,null,null,null,null,42.0,null
stddev,0.0,null,0.0,15327.745203645047,4.2577070010125135,19801.37667167314,2.86690185513737,0.34524635615691657,0.0,null,null,null,0.0,null,null,null,0.0,null,null,null,null,0.0,null,0.0,null,null,0.0,0.0,null,null,null,0.0,null,null,null,null,6.923845595257144,11.11815091092347,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null
min,0,null,0.0,905,0.2513888888888889,906,569,1,20180925,2018-09-21T00:00:00.0+04:00,2018-09-21T00:00:00.0+04:00,daily,1,null,null,null,15061,null,null,release,Windows_NT,5.1,null,2016,2018-09-21T04:39:03.372Z,null,20180925,20151216175450,43.0.1,release,null,240,Mozilla,google,null,null,0,0,null,null,null,null,RU,Chelyabinsk,CHE,ru,0635c7d4-069c-41f4-adf0-76d7a1039eca,13c73d37-14b5-48f5-986f-da1d7b6a4a7e,null,null,42,189d7038-5cfe-44a1-991c-f3d564d598cb
max,0,null,0.0,48933,13.5925,70472,580,2,20180925,2018-09-25T00:00:00.0+04:00,2018-09-25T00:00:00.0+04:00,shutdown,1,null,null,null,15061,null,null,release,Windows_NT,5.1,null,2016,2018-09-25T20:00:00.025Z,null,20180925,20151216175450,43.0.1,release,null,240,Mozilla,google,null,null,19,31,null,null,null,null,RU,Chelyabinsk,CHE,ru,ff148695-644e-4a33-83a8-8283873d734c,ceb10791-fe9e-4715-a3f2-850b71711abf,null,null,42,fb1978c3-436a-4b00-af5e-b2203e2722d1


In [20]:
# clients daily for client 9a4d - so why are there 619 active hours?
display(cd_1day.select(cd_cols).where("client_id like '%a749453b9a4d'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,0,619.9513888886801,819.223343,132,1.0,null,null,null,2011-03-28 00:00:00,null,release,Windows_NT,5.1,null,2016,null,12,20151216175450,null,release,true,true,240,Mozilla,true,google,null,16.056338028169016,25.760563380281692,null,null,RU,Chelyabinsk,CHE,ru,null,null,null,42


In [21]:
# One ping for client e81b with 0 active_ticks
display(ms_1day_ping.select(ms_ping_cols).where("client_id like '%ffb49892e81b'"))

uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_version,update_channel,update_enabled,update_auto_download,active_experiment_branch,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,telem_event_counts,tab_event_count,window_event_count,errors_collected_count,current_theme,formautofill_availability,media_page_count,country,city,geo_subdivision1,locale,antivirus,antispyware,firewall,session_id,subsession_id,sync_configured,sync_count_desktop,sync_count_mobile,disabled_addons_ids,active_theme,user_prefs,experiments,sample_id,document_id
0,null,0.0,86400,24.0,4519971,54,53,20180925,2018-08-03T00:00:00.0-05:00,2018-09-24T00:00:00.0-05:00,daily,0,null,null,null,17709,null,null,release,Windows_NT,6.2,null,2013,2018-09-25T05:00:00.015Z,null,20180925,20151216175450,43.0.1,release,true,true,null,-300,Mozilla,false,google,null,null,10,26,null,null,null,null,null,null,null,US,Chicago,IL,en-US,null,null,null,ca4b3f4a-16c6-4ff1-83ed-a322e16b13b2,eada7151-61b7-499d-95a7-492cd6db0344,null,null,null,null,"List({972ce4c6-7e08-4474-a285-3208198ce6fd}, false, Default, false, false, 43.0.1, 4, null, false, false, 17709, 17711, null, null, null, null)",null,null,42,2bc71816-e8e3-4877-8904-337ab255f9e4


In [22]:
# Clients daily for cient e81b - why are there 616 active hours?
display(cd_1day.select(cd_cols).where("client_id like '%ffb49892e81b'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,0,616.64722222222,24.0,0,0.0,null,null,null,2018-06-27 00:00:00,null,release,Windows_NT,6.2,null,2013,null,1,20151216175450,null,release,true,true,-300,Mozilla,false,google,null,10.0,26.0,null,null,US,Chicago,IL,en-US,null,null,null,42


Look at pings with high uri count  
  
** The active hours are higher than would be calculated from the sum of the active ticks**

In [24]:
# All pings for client 0c13 - 304 pings with all 0 active_ticks
display(ms_1day_ping.select(ms_ping_cols).where("client_id like '%4d8640730c13'").describe())

summary,uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_version,update_channel,active_experiment_branch,timezone_offset,vendor,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,tab_event_count,window_event_count,errors_collected_count,media_page_count,country,city,geo_subdivision1,locale,session_id,subsession_id,sync_count_desktop,sync_count_mobile,sample_id,document_id
count,304,0,304,304,304,304,304,304,304,304,304,304,304,304,304,304,304,0,0,304,304,304,0,304,304,304,304,304,304,304,0,304,304,304,10,304,292,292,304,225,0,0,304,304,304,304,304,304,0,0,304,304
mean,543.8651315789474,null,0.0,32816.70394736842,9.11575109649123,36822.59539473684,129.39802631578948,1.25,2.0180925E7,null,null,null,4.0,17.335526315789473,3.286184210526316,27.44078947368421,17588.0,null,null,null,null,6.100000000000016,null,2015.0,null,null,2.0180925E7,2.0170417065206E13,null,null,null,330.0,null,null,1.0,null,12.808219178082192,21826.130136986303,194.0559210526316,3.6222222222222222,null,null,null,null,null,null,null,null,null,null,42.0,null
stddev,296.24236058660347,null,0.0,9785.770644838542,2.718269623566261,7284.255764570261,58.23427247885234,0.4337266557190002,0.0,null,null,null,0.0,5.120906580921616,1.3201934722753017,19.992154749298255,0.0,null,null,null,null,0.0,null,0.0,null,null,0.0,0.0,null,null,null,0.0,null,null,0.0,null,6.3628162392726075,19933.249677649863,133.7183144583834,1.9965247584518235,null,null,null,null,null,null,null,null,null,null,0.0,null
min,7,null,0.0,26,0.007222222222222222,26,1,1,20180925,2018-02-27T00:00:00.0+05:30,2018-02-27T00:00:00.0+05:30,shutdown,4,2,1,1,17588,null,null,esr,Windows_NT,6.1,null,2015,2018-02-27T10:38:06.514Z,C3,20180925,20170417065206,52.1.0,esr,null,330,Mozilla,google,1,"Tue, 25 Sep 2018 03:45:20 GMT",0,0,1,1,null,null,US,Scottsdale,AZ,en-US,011d45bd-d9b7-4e7b-baaf-372dd9aa3717,05b15fa3-8f03-4408-befe-c23a7af26d4f,null,null,42,006515b9-4e4d-4136-b6a1-090d60df50b8
max,994,null,0.0,46713,12.975833333333334,65147,216,2,20180925,2018-09-24T00:00:00.0+05:30,2018-09-24T00:00:00.0+05:30,shutdown,4,39,8,89,17588,null,null,esr,Windows_NT,6.1,null,2015,2018-09-24T14:08:50.680Z,C3,20180925,20170417065206,52.1.0,esr,null,330,Mozilla,google-nocodes,1,"Tue, 25 Sep 2018 09:04:41 GMT",18,58886,407,11,null,null,US,Scottsdale,AZ,en-US,fd6a0d15-69d1-4484-a724-31075b839fb2,fe6003e0-127e-4ebe-9c24-ac60157b9fdd,null,null,42,ff739935-27d9-48f3-b17e-9d33e87e95e8


In [25]:
# Clients daily for client 0c13 - why are there 984 active hours?
display(cd_1day.select(cd_cols).where("client_id like '%4d8640730c13'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,165335,984.2125000002102,2771.188327,228,4.0,39,8,89,2018-02-26 00:00:00,null,esr,Windows_NT,6.1,null,2015,C3,156,20170417065206,52.1.0,esr,false,false,330,Mozilla,true,google,10,12.808219178082192,21826.130136986303,58993,815,US,Scottsdale,AZ,en-US,false,null,null,42


In [26]:
# All pings for client fa15 - there is a total of 3515 active ticks -- which should be 4.88 active hours
ms_1day_cfa15_ah = ms_1day_ping.where("client_id like '%dcf39efcfa15'")
display(ms_1day_cfa15_ah.select(ms_ping_cols))

uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,active_experiment_branch,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,telem_event_counts,tab_event_count,window_event_count,errors_collected_count,current_theme,formautofill_availability,media_page_count,country,city,geo_subdivision1,locale,antivirus,antispyware,firewall,session_id,subsession_id,sync_configured,sync_count_desktop,sync_count_mobile,disabled_addons_ids,active_theme,user_prefs,experiments,sample_id,document_id
0,0,0.0,26539,7.371944444444445,26539,2,1,20180925,2015-10-16T00:00:00.0-04:00,2015-10-16T00:00:00.0-04:00,shutdown,0,null,null,null,16206,null,null,release,Windows_NT,6.1,null,2014,2015-10-16T23:32:21.668Z,null,20180925,20150929144111,null,release,false,true,null,-240,Mozilla,false,yahoo,null,"Tue, 25 Sep 2018 16:17:13 GMT",12,25,null,null,null,null,null,null,null,US,New York,NY,en-US,null,null,null,20f40b01-05b3-468c-bf1b-702a750eff07,a8e24beb-9759-47b8-9428-472cc634acc9,null,null,null,null,"List({972ce4c6-7e08-4474-a285-3208198ce6fd}, false, Default, false, false, 41.0.1, 4, null, false, false, 16206, 16720, null, null, null, null)",null,null,42,edc4dc43-9080-4cfb-888f-863b2b150756
0,0,0.0,1500,0.4166666666666667,1531,3,1,20180925,2015-10-19T00:00:00.0-04:00,2015-10-19T00:00:00.0-04:00,shutdown,0,null,null,null,16206,null,null,release,Windows_NT,6.1,null,2014,2015-10-19T16:23:50.902Z,null,20180925,20150929144111,null,release,false,true,null,-240,Mozilla,false,yahoo,null,"Tue, 25 Sep 2018 16:17:13 GMT",12,null,null,null,null,null,null,null,null,US,New York,NY,en-US,null,null,null,dfb5c346-f976-44ad-b435-a111640a2155,a7219554-c853-4e92-81ce-bb5defc1a288,null,null,null,null,"List({972ce4c6-7e08-4474-a285-3208198ce6fd}, false, Default, false, false, 41.0.1, 4, null, false, false, 16206, 16720, null, null, null, null)",null,null,42,e1da84ab-b6eb-40ac-a76d-0a9d4bc2bd50
0,0,0.0,14170,3.936111111111111,14171,5,1,20180925,2015-10-20T00:00:00.0-04:00,2015-10-20T00:00:00.0-04:00,shutdown,0,null,null,null,16206,null,null,release,Windows_NT,6.1,null,2014,2015-10-20T20:01:36.739Z,null,20180925,20150929144111,null,release,false,true,null,-240,Mozilla,false,yahoo,null,"Tue, 25 Sep 2018 16:17:13 GMT",12,43,null,null,null,null,null,null,null,US,New York,NY,en-US,null,null,null,d5657398-7083-4fe6-9143-427f3309231f,a4f8edfa-b077-4e57-b3bc-211f4522dbb7,null,null,null,null,"List({972ce4c6-7e08-4474-a285-3208198ce6fd}, false, Default, false, false, 41.0.1, 4, null, false, false, 16206, 16720, null, null, null, null)",null,null,42,8a5f72c7-c570-49e9-abea-46064b385864
0,0,0.0,26525,7.368055555555555,26525,11,1,20180925,2015-10-23T00:00:00.0-04:00,2015-10-23T00:00:00.0-04:00,shutdown,0,null,null,null,16206,null,null,release,Windows_NT,6.1,null,2014,2015-10-23T23:22:33.629Z,null,20180925,20150929144111,null,release,false,true,null,-240,Mozilla,false,yahoo,null,"Tue, 25 Sep 2018 16:16:13 GMT",12,null,null,null,null,null,null,null,null,US,New York,NY,en-US,null,null,null,c4d87ec5-7f78-4d24-bc4b-9c0e18f6586a,e4d9e61f-5cb7-4e11-8f8e-e2e96a9f345f,null,null,null,null,"List({972ce4c6-7e08-4474-a285-3208198ce6fd}, false, Default, false, false, 41.0.1, 4, null, false, false, 16206, 16720, null, null, null, null)",null,null,42,594c3139-1f59-43fe-a1ee-380ba1df1e3b
0,0,0.0,9218,2.5605555555555557,9219,15,1,20180925,2015-10-27T00:00:00.0-04:00,2015-10-27T00:00:00.0-04:00,aborted-session,0,null,null,null,16206,null,null

In [27]:
ms_1day_cfa15_ah.select(F.sum('active_ticks')).show()

+-----------------+
sum(active_ticks)|
+-----------------+
 3515|
+-----------------+

In [28]:
# Clients daily for client fa15 - why are there 613 active hours?
display(cd_1day.select(cd_cols).where("client_id like '%dcf39efcfa15'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,20842,613.8236111112802,1024.274449,239,5.921875,9,2,42,2014-05-16 00:00:00,20180621121604,release,Windows_NT,6.1,7601,2014,null,243,20150929144111,52.0.1,release,false,true,-240,Mozilla,false,other-Search,null,1.9478260869565218,2550.1052631578946,850,26,US,New York,NY,en-US,false,null,null,42


In [29]:
# All pings for client 3776, 746 pings for a total of 65653 active ticks
ms_1day_c3776_ah = ms_1day_ping.where("client_id like '%418545233776'")
display(ms_1day_c3776_ah.select(ms_ping_cols).describe())

summary,uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_display_version,update_channel,active_experiment_branch,timezone_offset,vendor,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,tab_event_count,window_event_count,errors_collected_count,media_page_count,country,city,geo_subdivision1,locale,session_id,subsession_id,sync_count_desktop,sync_count_mobile,sample_id,document_id
count,746,746,746,746,746,746,746,746,746,746,746,746,746,746,746,673,746,0,183,746,746,746,89,746,746,0,746,746,746,746,0,746,746,746,15,746,440,440,654,346,0,0,746,746,746,746,746,746,0,0,746,746
mean,233.39544235924933,88.00670241286863,0.1222315311289842,17266.319034852546,4.796199731903481,19515.08310991957,227.07908847184987,1.1126005361930296,2.0180925E7,null,null,null,4.461126005361931,17.739946380697052,1.9611260053619304,16.306092124814263,17253.0,null,2.017331276370456E13,null,null,6.1000000000000005,7601.0,2016.0563002680965,null,null,2.0180925E7,2.0173992860971848E13,52.0,null,null,-254.39678284182307,null,null,0.8666666666666667,null,15.715909090909092,8929.438636363637,69.1223241590214,4.624277456647399,null,null,null,null,null,null,null,null,null,null,42.0,null
stddev,223.02543372131925,308.13723292947503,0.42796837906871515,12493.89973337967,3.4705277037165736,11765.182133169068,126.0505722190577,0.31631594722604606,0.0,null,null,null,1.3378198503111596,13.898845956528035,1.7903067316796055,12.090491210523187,0.0,null,4.322561367987885E9,null,null,0.0,0.0,0.8205705894488364,null,null,0.0,4.539069628110934E9,NaN,null,null,25.64021988092226,null,null,0.35186577527449836,null,4.754446099580094,6990.071892823489,55.896346507943676,6.016681176111011,null,null,null,null,null,null,null,null,null,null,0.0,null
min,0,0,0.0,4,0.0011111111111111111,5,4,1,20180925,2017-03-28T00:00:00.0-04:00,2017-03-28T00:00:00.0-04:00,aborted-session,0,1,1,1,17253,null,20170303022339,esr,Windows_NT,6.1,7601,2015,2017-03-29T02:47:57.816Z,null,20180925,20170303022339,52.0,esr,null,-300,Mozilla,google,0,"Tue, 25 Sep 2018 17:26:27 GMT",0,0,1,1,null,null,US,New York,NY,en-US,010557df-0132-475b-b1e5-5ac6430ef8be,00432f56-6675-44ad-ac08-6a69e1c3dbd0,null,null,42,00b7de71-1b03-4702-9186-58cb54a0292c
max,1680,2336,3.2444444444444445,36564,10.156666666666666,54294,462,2,20180925,2018-09-25T14:00:00.0-04:00,2018-09-25T14:00:00.0-04:00,shutdown,8,81,15,61,17253,null,20180621121604,esr,Windows_NT,6.1,7601,2018,2018-09-25T22:59:31.867Z,null,20180925,20180903060751,60.2.0esr,esr,null,-240,Mozilla,google-nocodes,1,"Tue, 25 Sep 2018 23:49:21 GMT",19,23531,369,35,null,null,US,New York,NY,en-US,ff5a819d-800f-4d00-afa5-7a3d67eaf140,fec7f360-d8b9-4d96-8f1f-cd20db170881,null,null,42,ff8c3001-554c-4cde-b2c7-5506942e3eed


In [30]:
ms_1day_c3776_ah.select(F.sum('active_ticks')).show()

+-----------------+
sum(active_ticks)|
+-----------------+
 65653|
+-----------------+

In [31]:
# Clients daily for client 3776 - 65,653 * 5 / 3600 should be 91.184 hours -- why are there 799 active hours?
display(cd_1day.select(cd_cols).where("client_id like '%418545233776'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,174113,799.4555555555197,3577.964996,662,4.461126005361931,81,15,61,2017-03-28 00:00:00,20170802111520,esr,Windows_NT,6.1,7601,2017,null,388,20170627155318,52.2.1,esr,false,false,-240,Mozilla,false,google-nocodes,13,15.715909090909092,8929.438636363637,45206,1600,US,New York,NY,en-US,false,null,null,42


In [32]:
# All pings for client c62d
ms_1day_cc62d_ah = ms_1day_ping.where("client_id like '%f2a01cedc62d'")
display(ms_1day_cc62d_ah.select(ms_ping_cols).describe())

summary,uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_display_version,update_channel,active_experiment_branch,timezone_offset,vendor,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,tab_event_count,window_event_count,errors_collected_count,media_page_count,country,city,geo_subdivision1,locale,session_id,subsession_id,sync_count_desktop,sync_count_mobile,sample_id,document_id
count,6627,6627,6627,6627,6627,6627,6627,6627,6627,6627,6627,6627,6627,561,561,508,6627,1075,117,6627,6627,6627,539,6627,6627,144,6627,6627,589,6627,0,6627,6627,6604,28,588,4177,4177,437,94,0,0,6627,6627,6607,6627,6627,6627,15,15,6627,6627
mean,6.9299834012373625,42.33740757507168,0.05880195496537731,25805.975403651726,7.168326501014371,69031.39278708314,391.07258186207935,1.511090991398823,2.0180925E7,null,null,null,1.0777123887128415,9.03921568627451,1.3101604278074865,4.43503937007874,16897.0,17540.028837209302,2.0173025547634215E13,null,null,6.100000000000001,7601.0,2017.173683416327,null,null,2.0180925E7,2.015338382713646E13,60.943661971830984,null,null,479.95473064735177,null,null,2.7857142857142856,null,30.823078764663634,13193.061288005745,44.391304347826086,3.8404255319148937,null,null,null,null,17.285714285714285,null,null,null,1.2666666666666666,0.0,42.0,null
stddev,49.26096065381184,283.7736685330598,0.39413009518480524,57128.04262229155,15.868900728414317,365729.0755016934,476.0036246314291,2.7068217923488405,0.0,null,null,null,3.229219200286234,13.002275711177171,0.7792304875891264,5.238466489551596,0.0,228.26696715120235,1.2658307441394485E10,null,null,0.0,0.0,0.859341733117977,null,null,0.0,8.006171578033172E9,2.6735284646391393,null,null,2.2108320440334377,null,null,3.3813994785979418,null,78.95692156747029,19348.50608496433,74.05623462960398,7.568047135911886,null,null,null,null,20.81947524436073,null,null,null,0.4577377082170634,0.0,0.0,null
min,0,0,0.0,0,0.0,0,1,1,20180925,2016-04-06T00:00:00.0+08:00,2016-04-06T00:00:00.0+08:00,aborted-session,0,1,1,1,16897,16932,20150917150946,beta,Windows_NT,6.1,7601,2004,2016-04-06T09:37:48.447Z,MozillaOnline,20180925,20150917150946,47.0.1,beta,null,420,Mozilla,baidu,0,"Tue, 25 Sep 2018 00:06:09 GMT",0,0,1,1,null,null,CN,??,01,en-US,0014ab84-381c-46b4-af55-42aae282fe6c,000e77fb-2a28-4ac0-bd37-021f990eb803,1,0,42,00063150-42fd-4ca4-bbdd-fa4f42c0765e
max,1741,5194,7.213888888888889,2165662,601.5727777777778,11428882,6597,99,20180925,2018-09-26T00:00:00.0+08:00,2018-09-26T00:00:00.0+08:00,shutdown,26,215,8,69,16897,17799,20180920131237,release,Windows_NT,6.1,7601,2018,2018-09-25T23:30:35.551Z,MozillaOnline,20180925,20180920131237,62.0.2,release,null,540,Mozilla,google-nocodes,14,"Tue, 25 Sep 2018 18:41:37 GMT",4528,133957,616,60,null,null,US,Ürümqi,ZJ,zh-CN,ffefd3ba-6ef9-432d-a4fc-3f7d4865603e,ffea2623-e171-4669-a628-107cc4ef9c77,2,0,42,ffe5cca7-9fab-43f4-9b4e-a6e5a989215e


In [33]:
ms_1day_cc62d_ah.select(F.sum('active_ticks')).show()

+-----------------+
sum(active_ticks)|
+-----------------+
 280570|
+-----------------+

In [34]:
# Clients daily for client c62d - 280,570 * 5 / 3600 should be 389.68 hours
display(cd_1day.select(cd_cols).where("client_id like '%f2a01cedc62d'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,45925,14137.222222222863,47504.499734,5929,1.0777123887128415,215,8,69,2016-04-06 00:00:00,20180323154952,release,Windows_NT,6.1,7601,2017,MozillaOnline,6574,20150917150946,62.0,release,false,false,480,Mozilla,true,baidu,78,30.823078764663634,13193.061288005745,19399,361,CN,Nanjing,JS,zh-CN,true,19,0,42


Look at pings with regular uri count

In [36]:
# One ping for client 6106 with 0 active ticks
display(ms_1day_ping.select(ms_ping_cols).where("client_id like '%c7b4cf906106'"))

uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,active_experiment_branch,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,telem_event_counts,tab_event_count,window_event_count,errors_collected_count,current_theme,formautofill_availability,media_page_count,country,city,geo_subdivision1,locale,antivirus,antispyware,firewall,session_id,subsession_id,sync_configured,sync_count_desktop,sync_count_mobile,disabled_addons_ids,active_theme,user_prefs,experiments,sample_id,document_id
28,0,0.0,86405,24.00138888888889,3597000,1311,42,20180925,2018-08-15T00:00:00.0+03:00,2018-09-25T00:00:00.0+03:00,daily,8,1,1,null,16006,null,null,release,Windows_NT,5.1,null,2013,2018-09-25T21:00:25.973Z,null,20180925,20170118123726,51.0,release,false,false,null,180,Mozilla,true,google-nocodes,null,"Tue, 25 Sep 2018 21:36:40 GMT",null,null,null,null,null,null,null,null,null,RU,Rostov-on-Don,ROS,ru,null,null,null,cb1d7a91-bf27-4d6a-8389-d178b121deff,e4e53da4-29e3-473a-b31a-fbbed9871d3f,null,null,null,null,"List({972ce4c6-7e08-4474-a285-3208198ce6fd}, false, Стандартная, false, false, 51.0, 4, null, false, false, 16006, 17185, null, null, null, null)",null,null,42,62bf8c8c-d868-465a-9827-b2a573f5f77f


In [37]:
# Clients daily for client 6106 - why 502 active hours?
display(cd_1day.select(cd_cols).where("client_id like '%c7b4cf906106'"))

submission_date_s3,uri_count_sum,active_hours_sum,subsession_hours_sum,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,tab_event_count_sum,window_event_count_sum,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop_sum,sync_count_mobile_sum,sample_id
20180925,28,502.55416666667,24.001389,0,8.0,1,1,null,2013-10-28 00:00:00,null,release,Windows_NT,5.1,null,2013,null,1,20170118123726,51.0,release,false,false,180,Mozilla,true,google-nocodes,null,null,null,null,null,RU,Rostov-on-Don,ROS,ru,null,null,null,42


### Duplicate pings?

Look at pings with the same client_id and subsession_id, or the same client_id and document_id

In [39]:
# Get count of duplicates by client_id and subsession_id
ms_dup_ssid = ms_1day_ping.groupby('client_id', 'subsession_id').count().filter('count > 1')
ms_dup_ssid.select(F.sum('count')).show()

+----------+
sum(count)|
+----------+
 424235|
+----------+

In [40]:
# Get count of duplicates by client_id and document_id - same number as above
ms_dup_docid = ms_1day_ping.groupby('client_id', 'document_id').count().filter('count > 1')
ms_dup_ssid.select(F.sum('count')).show()

+----------+
sum(count)|
+----------+
 424235|
+----------+

In [41]:
# Get list of client_ids which may have duplicate pings
ms_1day_dups = ms_1day_ping.groupby('client_id', 'subsession_id').count().filter('count > 1').sort('client_id')
display(ms_1day_dups['subsession_id', 'count'])

subsession_id,count
3eb0f5ba-5cd4-41d9-b778-67ef06534f3c,2
09b10e21-042c-48d5-8b20-05777d063390,2
3117100d-699c-440d-913c-9916ea38ca61,2
fd03bafc-7dff-4ae8-80d4-ae9f19cc93e2,2
6bf7db32-5921-4b2f-9feb-d2cf84df20f9,2
d2f61463-df36-453f-9c37-588d5d7af4ac,2
466bcf27-6598-4668-839a-2f9580165fda,2
b1f304de-3f16-4f00-b8f0-699ede8c3748,3
e75ee704-a20e-44d5-b63c-7f97c59b24f4,3
45e5697b-21b2-43f6-963e-639f2e759c5b,2


These look like duplicate pings

In [43]:
# Look at client_id and subsesion_id with 19 pings - sometimes diffeent client_submission_date
display(ms_1day_ping['subsession_id', 'document_id', 'uri_count', 'active_ticks', 'active_hours', 'subsession_hours', 
                     'profile_subsession_counter', 'submission_date_s3', 'client_submission_date', 'subsession_start_date', 'reason',
                     'active_addons_count', 'tab_count', 'window_count', 'country', 'city'] \
        .where("client_id like '%65202d56e8df' AND subsession_id = '8f7447a1-8bf0-4880-ac06-20d4d4582503'"))

subsession_id,document_id,uri_count,active_ticks,active_hours,subsession_hours,profile_subsession_counter,submission_date_s3,client_submission_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,country,city
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:46:31 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:46:31 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:42:06 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:46:31 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:46:31 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:46:31 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:54:41 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:46:31 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:54:41 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana
8f7447a1-8bf0-4880-ac06-20d4d4582503,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0,11,37,0.05138888888888889,0.1688888888888889,449,20180925,"Tue, 25 Sep 2018 03:46:31 GMT",2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,KZ,Astana


In [44]:
display(ms_1day_ping.select(ms_ping_cols) \
        .where("client_id like '%65202d56e8df' AND subsession_id = '8f7447a1-8bf0-4880-ac06-20d4d4582503'") \
        .describe())

summary,uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_display_version,update_channel,active_experiment_branch,timezone_offset,vendor,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,tab_event_count,window_event_count,errors_collected_count,media_page_count,country,city,geo_subdivision1,locale,session_id,subsession_id,sync_count_desktop,sync_count_mobile,sample_id,document_id
count,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,0,0,19,19,19,19,19,19,0,19,19,19,19,0,19,19,19,0,19,0,0,0,0,0,0,19,19,19,19,19,19,0,0,19,19
mean,11.0,37.0,0.05138888888888891,608.0,0.16888888888888898,610.0,449.0,1.0,2.0180925E7,null,null,null,8.0,1.0,1.0,1.0,17547.0,null,null,null,null,6.099999999999998,7601.0,2018.0,null,null,2.0180925E7,2.0180830143136E13,62.0,null,null,360.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,42.0,null
stddev,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,0.0,0.0,0.0,0.0,0.0,null,null,null,null,0.0,0.0,0.0,null,null,0.0,0.0,0.0,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null
min,11,37,0.05138888888888889,608,0.1688888888888889,610,449,1,20180925,2018-09-24T19:00:00.0+06:00,2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,1,17547,null,null,release,Windows_NT,6.1,7601,2018,2018-09-24T13:25:07.067Z,null,20180925,20180830143136,62.0,release,null,360,Mozilla,yandex-ru,null,"Tue, 25 Sep 2018 03:39:55 GMT",null,null,null,null,null,null,KZ,Astana,AST,ru,289bd13d-7cf8-4a21-b8fd-16b0db986dab,8f7447a1-8bf0-4880-ac06-20d4d4582503,null,null,42,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0
max,11,37,0.05138888888888889,608,0.1688888888888889,610,449,1,20180925,2018-09-24T19:00:00.0+06:00,2018-09-24T19:00:00.0+06:00,shutdown,8,1,1,1,17547,null,null,release,Windows_NT,6.1,7601,2018,2018-09-24T13:25:07.067Z,null,20180925,20180830143136,62.0,release,null,360,Mozilla,yandex-ru,null,"Tue, 25 Sep 2018 04:10:46 GMT",null,null,null,null,null,null,KZ,Astana,AST,ru,289bd13d-7cf8-4a21-b8fd-16b0db986dab,8f7447a1-8bf0-4880-ac06-20d4d4582503,null,null,42,499e0dba-9d24-4f67-ad28-f5cac9ca8ba0


These look like duplicate pings, but the country and city are different.  How is this possible?

In [46]:
# Look at client_id and subsesion_id with 21 pings
display(ms_1day_ping['subsession_id', 'document_id', 'uri_count', 'active_ticks', 'active_hours', 'subsession_hours',
                     'profile_subsession_counter', 'submission_date_s3', 'client_submission_date', 'subsession_start_date', 'reason', 
                     'active_addons_count', 'tab_count', 'window_count', 'country', 'city'] \
        .where("client_id like '%455d13aec41f' AND subsession_id = '447de96d-89e5-41cf-914c-1683b37b8b2d'"))

subsession_id,document_id,uri_count,active_ticks,active_hours,subsession_hours,profile_subsession_counter,submission_date_s3,client_submission_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,country,city
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,GB,London
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,US,New York
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,US,Ashburn
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,AU,??
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,US,Phoenix
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,US,Dallas
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,US,Atlanta
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,GB,London
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,AU,??
447de96d-89e5-41cf-914c-1683b37b8b2d,56cf5194-6876-4075-8057-038a2d41fd62,0,0,0.0,0.016666666666666666,30315,20180925,null,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,US,New York


In [47]:
display(ms_1day_ping.select(ms_ping_cols) \
        .where("client_id like '%455d13aec41f' AND subsession_id = '447de96d-89e5-41cf-914c-1683b37b8b2d'") \
        .describe())

summary,uri_count,active_ticks,active_hours,subsession_length,subsession_hours,session_length,profile_subsession_counter,subsession_counter,submission_date_s3,session_start_date,subsession_start_date,reason,active_addons_count,tab_count,window_count,domains_count,profile_creation_date,profile_reset_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,creation_date,distribution_id,submission_date,app_build_id,app_display_version,update_channel,active_experiment_branch,timezone_offset,vendor,default_search_engine,devtools_toolbox_opened_count,client_submission_date,places_bookmarks_count,places_pages_count,tab_event_count,window_event_count,errors_collected_count,media_page_count,country,city,geo_subdivision1,locale,session_id,subsession_id,sync_count_desktop,sync_count_mobile,sample_id,document_id
count,21,21,21,21,21,21,21,21,21,21,21,21,21,0,0,0,21,0,0,21,21,21,0,21,21,0,21,21,0,21,0,21,21,21,0,0,0,0,0,0,0,0,21,21,20,21,21,21,0,0,21,21
mean,0.0,0.0,0.0,60.0,0.016666666666666666,64.0,30315.0,1.0,2.0180925E7,null,null,null,0.0,null,null,null,17489.0,null,null,null,null,5.099999999999999,null,2013.0,null,null,2.0180925E7,2.015121617545E13,null,null,null,420.0,null,null,null,null,null,null,null,null,null,null,null,null,18.0,null,null,null,null,null,42.0,null
stddev,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,0.0,null,null,null,0.0,null,null,null,null,0.0,null,0.0,null,null,0.0,0.0,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,NaN,null,null,null,null,null,0.0,null
min,0,0,0.0,60,0.016666666666666666,64,30315,1,20180925,2018-09-25T00:00:00.0+07:00,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,null,17489,null,null,release,Windows_NT,5.1,null,2013,2018-09-25T08:13:43.792Z,null,20180925,20151216175450,null,release,null,420,Mozilla,google,null,null,null,null,null,null,null,null,AU,??,18,en-US,3f7508a2-bf30-457e-a761-036d78925c2c,447de96d-89e5-41cf-914c-1683b37b8b2d,null,null,42,56cf5194-6876-4075-8057-038a2d41fd62
max,0,0,0.0,60,0.016666666666666666,64,30315,1,20180925,2018-09-25T00:00:00.0+07:00,2018-09-25T00:00:00.0+07:00,aborted-session,0,null,null,null,17489,null,null,release,Windows_NT,5.1,null,2013,2018-09-25T08:13:43.792Z,null,20180925,20151216175450,null,release,null,420,Mozilla,google,null,null,null,null,null,null,null,null,US,Phoenix,VA,en-US,3f7508a2-bf30-457e-a761-036d78925c2c,447de96d-89e5-41cf-914c-1683b37b8b2d,null,null,42,56cf5194-6876-4075-8057-038a2d41fd62


Removing the duplicate pings changes the mean and stddev, but not much

In [49]:
# Drop the duplicates
ms_1day_dedup_ping = ms_1day_ping.drop_duplicates(['client_id', 'subsession_id'])

In [50]:
o_count = ms_1day_ping.count()
d_count = ms_1day_dedup_ping.count()
dup_count = o_count - d_count
dup_perct = (dup_count * 1.0 / o_count) * 100

In [51]:
print 'Originally had ', o_count, ' records'
print 'Reduced to ', d_count, ' records'
print 'Removed ', dup_count, ' records which is', '%0.2f' % (dup_perct), '%'

Originally had 3982350 records
Reduced to 3643792 records
Removed 338558 records which is 8.50 %

In [52]:
# Regenerate the aggregate daily values without the duplicates
ms_1day_df_sum = ms_1day_dedup_ping.groupBy('client_id', 'submission_date_s3') \
      .agg(F.sum('uri_count').alias('td_uri'), \
           F.sum('active_ticks').alias('td_active_ticks'), \
           F.sum('active_hours').alias('td_active_hours'), \
           F.sum('subsession_hours').alias('td_subsession_hours'), \
           F.sum(F.when(F.col('subsession_counter') == 1, 1).otherwise(0)).alias('sessions_started_on_this_day'), \
           F.mean('active_addons_count').alias('active_addons_count_mean'), \
           F.max('tab_count').alias('tab_count_max'), \
           F.max('window_count').alias('window_count_max'), \
           F.max('domains_count').alias('domains_count_max'), \
           F.first('profile_creation_date').alias('profile_creation_date'), \
           F.first('previous_build_id').alias('previous_build_id'), \
           F.first('normalized_channel').alias('normalized_channel'), \
           F.first('os').alias('os'), \
           F.first('normalized_os_version').alias('normalized_os_version'), \
           F.first('windows_build_number').alias('windows_build_number'), \
           F.first('install_year').alias('install_year'), \
           F.first('distribution_id').alias('distribution_id'), \
           F.countDistinct('document_id').alias('pings_aggregated_by_this_row'), \
           F.first('app_build_id').alias('app_build_id'), \
           F.first('app_version').alias('app_version'), \
           F.first('update_channel').alias('update_channel'), \
           F.first('update_enabled').alias('update_enabled'), \
           F.first('update_auto_download').alias('update_auto_download'), \
           F.first('timezone_offset').alias('timezone_offset'), \
           F.first('vendor').alias('vendor'), \
           F.first('is_default_browser').alias('is_default_browser'), \
           F.first('default_search_engine').alias('default_search_engine'), \
           F.sum('devtools_toolbox_opened_count').alias('devtools_toolbox_opened_count_sum'), \
           F.mean('places_bookmarks_count').alias('places_bookmarks_count_mean'), \
           F.mean('places_pages_count').alias('places_pages_count_mean'), \
           F.sum('tab_event_count').alias('td_tab_event_count'), \
           F.sum('window_event_count').alias('td_window_event_count'), \
           F.first(F.when(F.col('country') == '??', None).otherwise(F.col('country'))).alias('country'), \
           F.first(F.when(F.col('country') == '??', None).otherwise(F.when(F.col('city') == None, '??') \
                                                                        .otherwise(F.col('city')))).alias('city'), \
           F.first('geo_subdivision1').alias('geo_subdivision1'), \
           F.first('locale').alias('locale'), \
           F.first('sync_configured').alias('sync_configured'), \
           F.sum('sync_count_desktop').alias('sync_count_desktop'), \
           F.sum('sync_count_mobile').alias('sync_count_mobile'), \
           F.first('sample_id').alias('sample_id'))
display(ms_1day_df_sum.select(ms_sum_cols))

submission_date_s3,td_uri,td_active_ticks,td_active_hours,td_subsession_hours,sessions_started_on_this_day,active_addons_count_mean,tab_count_max,window_count_max,domains_count_max,profile_creation_date,previous_build_id,normalized_channel,os,normalized_os_version,windows_build_number,install_year,distribution_id,pings_aggregated_by_this_row,app_build_id,app_display_version,update_channel,update_enabled,update_auto_download,timezone_offset,vendor,is_default_browser,default_search_engine,devtools_toolbox_opened_count_sum,places_bookmarks_count_mean,places_pages_count_mean,td_tab_event_count,td_window_event_count,country,city,geo_subdivision1,locale,sync_configured,sync_count_desktop,sync_count_mobile,sample_id
20180925,27,59,0.08194444444444444,23.99888888888889,0,14.0,21,1,2,17558,20180807170231,release,Windows_NT,6.3,9600,2018,null,3,20180830143136,62.0,release,true,true,120,Mozilla,false,google,null,65.0,12005.0,1,null,FR,Lyon,ARA,en-US,null,null,null,42
20180925,17,137,0.19027777777777777,0.43944444444444447,2,9.5,1,1,2,17684,null,release,Windows_NT,6.1,7601,2018,null,2,20180830143136,62.0,release,true,true,210,Mozilla,true,google,null,8.0,827.0,null,null,IR,??,null,en-US,null,null,null,42
20180925,0,180,0.25,0.31444444444444447,4,10.0,3,1,null,17714,null,release,Windows_NT,10.0,17134,2018,null,4,20180920131237,62.0.2,release,true,true,120,Mozilla,true,other-Startpage HTTPS,null,null,null,5,null,DE,Hamburg,HH,de,null,null,null,42
20180925,32,0,0.0,4.384444444444444,7,4.0,2,2,2,16245,null,esr,Windows_NT,5.2,null,2018,null,7,20180621064021,52.9.0,esr,true,true,300,Mozilla,true,google,null,15.0,528.0,null,null,PK,Lahore,PB,en-US,false,null,null,42
20180925,835,2039,2.8319444444444444,4.078055555555555,2,7.0,3,3,1,17588,20180917143811,beta,Windows_NT,6.3,9600,2016,null,2,20180920135444,63.0b8,beta,true,true,-420,Mozilla,false,google,null,11.0,256.0,6,6,IN,Jodhpur,RJ,en-US,null,null,null,42
20180925,0,3308,4.594444444444444,40.18472222222222,0,26.0,21,3,null,17173,null,release,Windows_NT,10.0,17134,2018,null,2,20180830143136,62.0,release,true,true,120,Mozilla,true,qwant,null,754.0,451.0,129,22,FR,??,ARA,fr,null,null,null,42
20180925,3,29,0.04027777777777778,0.6077777777777778,1,11.0,2,1,1,17257,null,release,Linux,4.4.0,null,null,canonical,2,20180912143528,62.0,release,false,true,-180,Mozilla,false,google,null,null,null,null,null,CL,Port Montt,LL,es-CL,null,null,null,42
20180925,9,0,0.0,1.9880555555555555,2,4.0,2,1,3,16230,null,esr,Windows_NT,5.1,null,2013,null,2,20180621064021,52.9.0,esr,true,true,180,Mozilla,true,google,null,36.0,41779.0,4,null,RO,Caransebes,CS,ro,false,null,null,42
20180925,0,0,0.0,2.741388888888889,9,0.0,null,null,null,17595,null,release,Windows_NT,5.1,null,2012,null,9,20150917150946,null,release,false,false,120,Mozilla,false,google,null,18.0,1023.0,null,null,FR,??,ARA,fr,null,null,null,42
20180925,43,358,0.49722222222222223,1.6788888888888889,3,9.0,3,2,6,16568,null,release,Windows_NT,10.0,17134,2018,null,4,20180920131237,62.0.2,release,true,false,240,Mozilla,false,google,null,null,null,3,1,RE,??,null,fr,null,null,null,42


In [53]:
# Summary stats for the deduped dataframe - this doesn't change the values much
display(ms_1day_df_sum['submission_date_s3', 'td_uri', 'td_active_hours', 'td_subsession_hours'].describe())

summary,submission_date_s3,td_uri,td_active_hours,td_subsession_hours
count,1045624,1045624,1045624,1045624
mean,2.0180925E7,131.21150241386962,0.6409368964263314,8.78370616918171
stddev,0.0,4548.256718640441,1.4325052334482906,73.07073876045386
min,20180925,0,0.0,0.0
max,20180925,4530092,436.9305555555554,47265.95277777775


In [54]:
# Summary stats for agg of all pings
display(ms_1day_sum['submission_date_s3', 'td_uri', 'td_active_hours', 'td_subsession_hours'].describe())

summary,submission_date_s3,td_uri,td_active_hours,td_subsession_hours
count,1045624,1045624,1045624,1045624
mean,2.0180925E7,131.88279438880517,0.6433114580384507,8.846028020641818
stddev,0.0,4553.284675465035,1.465108961921599,74.1812457976852
min,20180925,0,0.0,0.0
max,20180925,4530092,436.93055555555543,47504.49972222209
